<a href="https://colab.research.google.com/github/jiin124/2021_2_SAI_ML/blob/main/10%EC%A3%BC%EC%B0%A8/%EB%AC%B8%EC%84%9C%EA%B5%B0%EC%A7%91%ED%99%94_%EC%86%8C%EA%B0%9C%EC%99%80_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import glob, os

path=r'/content/drive/MyDrive/topics'
all_files=glob.glob(os.path.join(path,"*.data"))
filename_list=[]
opinion_text=[]

#개별 파일의 파일명은 filename_list로 취합
#개별 파일의 파일 내용은 dataframe 로딩 후 다시 string으로 변환해 opnion_text list로 취합
for file_ in all_files:
  #개별 파일을 읽어서 dataframe으로 생성
  df=pd.read_table(file_,index_col=None,header=0,encoding='latin1')

  filename_=file_.split('/')[-1]
  filename=filename_.split('.')[0]

  #파일명 list와 파일 내용 list에 파일명과 파일 내용을 추가
  filename_list.append(filename)
  opinion_text.append(df.to_string())

#파일명 list와 파일 내용 list 객체를 dataframe으로 설정
document_df=pd.DataFrame({'filename':filename_list,'opinion_text':opinion_text})
document_df.head()

,filename,opinion_text
0,battery-life_ipod_nano_8gb,...
1,features_windows7,...
2,display_garmin_nuvi_255W_gps,...
3,keyboard_netbook_1005ha,...
4,accuracy_garmin_nuvi_255W_gps,...


In [2]:
from nltk.stem import WordNetLemmatizer
import nltk
import string


remove_punct_dict=dict((ord(punct),None) for punct in string.punctuation)
lemmar=WordNetLemmatizer()


def LemTokens(tokens):
  return [lemmar.lemmatize(token) for token in tokens]

def LemNormalize(text):
  return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect=TfidfVectorizer(tokenizer=LemNormalize,stop_words='english',
                           ngram_range=(1,2),min_df=0.05,max_df=0.85)

#opinion_text 칼럼값으로 피처 벡터화 수행
feature_vect=tfidf_vect.fit_transform(document_df['opinion_text'])

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  % sorted(inconsistent)


In [6]:
from sklearn.cluster import KMeans

#5개의 집합으로 군집화 수행
km_cluster=KMeans(n_clusters=5,max_iter=10000,random_state=0)
km_cluster.fit(feature_vect)
cluster_label=km_cluster.labels_
cluster_centers=km_cluster.cluster_centers_

In [7]:
document_df['cluster_label']=cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,battery-life_ipod_nano_8gb,...,2
1,features_windows7,...,0
2,display_garmin_nuvi_255W_gps,...,0
3,keyboard_netbook_1005ha,...,2
4,accuracy_garmin_nuvi_255W_gps,...,0


In [8]:
document_df[document_df['cluster_label']==0].sort_values(by='filename')

,filename,opinion_text,cluster_label
4,accuracy_garmin_nuvi_255W_gps,...,0
14,buttons_amazon_kindle,...,0
5,directions_garmin_nuvi_255W_gps,...,0
2,display_garmin_nuvi_255W_gps,...,0
20,eyesight-issues_amazon_kindle,...,0
1,features_windows7,...,0
10,fonts_amazon_kindle,...,0
12,navigation_amazon_kindle,...,0


In [9]:
document_df[document_df['cluster_label']==1].sort_values(by='filename')

,filename,opinion_text,cluster_label
25,comfort_honda_accord_2008,...,1
11,comfort_toyota_camry_2007,...,1
7,gas_mileage_toyota_camry_2007,...,1
27,interior_honda_accord_2008,...,1
17,interior_toyota_camry_2007,...,1
28,mileage_honda_accord_2008,...,1
22,performance_honda_accord_2008,...,1
29,quality_toyota_camry_2007,...,1


In [10]:
document_df[document_df['cluster_label']==2].sort_values(by='filename')

,filename,opinion_text,cluster_label
18,battery-life_amazon_kindle,...,2
0,battery-life_ipod_nano_8gb,...,2
6,battery-life_netbook_1005ha,...,2
3,keyboard_netbook_1005ha,...,2
16,performance_netbook_1005ha,...,2


In [11]:
from sklearn.cluster import KMeans

#3개의 집합으로 군집화 수행
km_cluster=KMeans(n_clusters=3,max_iter=10000,random_state=0)
km_cluster.fit(feature_vect)
cluster_label=km_cluster.labels_
cluster_centers=km_cluster.cluster_centers_

In [12]:
document_df['cluster_label']=cluster_label
document_df.sort_values(by='cluster_label')

,filename,opinion_text,cluster_label
26,parking_bestwestern_hotel_sfo,...,0
24,free_bestwestern_hotel_sfo,...,0
23,location_bestwestern_hotel_sfo,...,0
21,bathroom_bestwestern_hotel_sfo,...,0
19,location_holiday_inn_london,...,0
8,price_holiday_inn_london,...,0
9,food_holiday_inn_london,...,0
15,food_swissotel_chicago,...,0
0,battery-life_ipod_nano_8gb,...,1
20,eyesight-issues_amazon_kindle,...,1


### 군집별 핵심단어 추출

In [13]:
cluster_centers=km_cluster.cluster_centers_
print('cluster_centers shape:',cluster_centers.shape)
print(cluster_centers)

cluster_centers shape: (3, 5125)
[[0.         0.00163644 0.0060297  ... 0.00039059 0.         0.00256806]
 [0.01261191 0.         0.00204708 ... 0.         0.006166   0.        ]
 [0.         0.0010902  0.00098759 ... 0.00147266 0.         0.        ]]


군집이 3개, word피처가 5125개로 구성되었음을 의미 각 행의 배열 값은 각 군집내의 5125피처의 위치가 개별 중심과 얼마나 가까운가를 상대 값으로 나타낸 것. 0에서 1까지의 값을 가질 수 있으며 1에 가까울수록 중심과 가까운 값을 의미

In [27]:
# 군집별 top n 핵심단어, 그 단어의 중심 위치 상대값, 대상 제목들을 반환함.
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num, top_n_features=10):
    cluster_details = {}

    # cluster_centers array 의 값이 큰 순으로 정렬된 index 값을 반환
    # 군집 중심점(centroid)별 할당된 word 피처들의 거리값이 큰 순으로 값을 구하기 위함.
    centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:, ::-1]

    # 개별 군집별로 iteration하면서 핵심단어, 그 단어의 중심 위치 상대값, 대상 제목 입력
    for cluster_num in range(clusters_num):
        # 개별 군집별 정보를 담을 데이터 초기화.
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num

        # cluster_centers_.argsort()[:,::-1] 로 구한 index 를 이용하여 top n 피처 단어를 구함.
        top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
        top_features = [feature_names[ind] for ind in top_feature_indexes]

        # top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함
        top_feature_values = cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()

        # cluster_details 딕셔너리 객체에 개별 군집별 핵심 단어와 중심위치 상대값, 그리고 해당 파일명 입력
        cluster_details[cluster_num]['top_features'] = top_features
        cluster_details[cluster_num]['top_features_value'] = top_feature_values
        filenames = cluster_data[cluster_data['cluster_label'] == cluster_num]['filename']
        filenames = filenames.values.tolist()
        cluster_details[cluster_num]['filenames'] = filenames

    return cluster_details

In [28]:
def print_cluster_details(cluster_details):
    for cluster_num, cluster_detail in cluster_details.items():
        print('####### Cluster {0}'.format(cluster_num))
        print('Top features:', cluster_detail['top_features'])
        print('filename :', cluster_detail['filenames'][:7])
        print('==================================================')

In [29]:
feature_names = tfidf_vect.get_feature_names()
cluster_details =get_cluster_details(cluster_model=km_cluster, cluster_data=document_df, feature_names=feature_names, clusters_num=3, top_n_features=10)

print_cluster_details(cluster_details)

####### Cluster 0
Top features: ['food', 'hotel', 'location', 'bathroom', 'parking', 'room', 'price', 'free', 'great location', 'wine']
filename : ['price_holiday_inn_london', 'food_holiday_inn_london', 'food_swissotel_chicago', 'location_holiday_inn_london', 'bathroom_bestwestern_hotel_sfo', 'location_bestwestern_hotel_sfo', 'free_bestwestern_hotel_sfo']
####### Cluster 1
Top features: ['battery', 'life', 'battery life', 'kindle', 'direction', 'button', 'keyboard', 'page', 'book', 'font']
filename : ['battery-life_ipod_nano_8gb', 'features_windows7', 'display_garmin_nuvi_255W_gps', 'keyboard_netbook_1005ha', 'accuracy_garmin_nuvi_255W_gps', 'directions_garmin_nuvi_255W_gps', 'battery-life_netbook_1005ha']
####### Cluster 2
Top features: ['interior', 'mileage', 'comfortable', 'gas mileage', 'gas', 'car', 'seat', 'performance', 'quality', 'comfort']
filename : ['gas_mileage_toyota_camry_2007', 'comfort_toyota_camry_2007', 'interior_toyota_camry_2007', 'performance_honda_accord_2008', 'c

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
